# FRF_Currituck Sound Current Data

Reading the FRF Thredds Server netCDF data for horizintal water currents recorded in the Currituck Sound and writing these data, or a subset thereof, to a comma-separated values file. 

What data?

- time (converted to ISO 8601 format)
- meanPressure: sea water pressure
- aveE: verically averaged eastward water current
- aveN: vertically averaged northward water current
- currentSpeed: vertically averaged current speed
- currentDirection: vertically averaged current direction

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import netCDF4

%matplotlib inline

### Load source file location information and load copy of data to local container:

In [83]:
url_base='https://chlthredds.erdc.dren.mil/thredds/dodsC/frf/oceanography/currents/'
url='CS05-ADOP/CS05-ADOP.ncml'
urls=[
    'CS01-ADOP/CS01-ADOP.ncml',
    'CS02-ADOP/CS02-ADOP.ncml',
    'CS03-ADOP/CS03-ADOP.ncml',
    'CS04-ADOP/CS04-ADOP.ncml',
    'CS05-ADOP/CS05-ADOP.ncml']
    
nc=netCDF4.Dataset(url_base+url)

### Validate Variables:

Check for variables present in the data, along with their length. If the lengths differ (indicating missing data) it's probably best to just drop the variable from further consideration. If you really need it, you might be able to pad it to match the size of the other vars...

In [84]:
print('Variables present in the file, and their length (based on np.shape:')
for var in nc.variables:
    print(var, np.shape(nc.variables[var]))

Variables present in the file, and their length (based on np.shape:
latitude ()
longitude ()
depth (15,)
numBins ()
cellSize ()
time (32390,)
meanPressure (32390,)
aveE (32390,)
aveN (32390,)
blankDist (32390,)
maxCell (32390,)
currentSpeed (32390,)
currentDirection (32390,)
qcFlag (32390,)
meanIntensity (32390,)
aveTime (32390,)
currentEast (32390, 15)
currentNorth (32390, 15)
currentUp (32390, 15)
beam1Intensity (32390, 15)
beam2Intensity (32390, 15)
beam3Intensity (32390, 15)


### List the Station Position (Latitude and Longitude):

In [85]:
print(nc.variables['longitude'][0], nc.variables['latitude'][0])

-75.76118 36.178999


### Copy Data to Comma-Separated Values file:

In [86]:
times=nc.variables['time']

csdict={
'meanPressure' : nc.variables['meanPressure'],
'aveE' : nc.variables['aveE'],
'aveN' : nc.variables['aveN'],
'currentSpeed' : nc.variables['currentSpeed'],
'currentDirection' : nc.variables['currentDirection']
}
    
dt = netCDF4.num2date(times[:],times.units)
df = pd.DataFrame(csdict,index=dt)
df.to_csv('./FRF_'+url[10:-5]+'_Currents_ALL.csv')

## Parse current data gathered by station into individual files by year and month...

In [6]:
file_path='/Users/paulp/GoogleDrive/projects/CurrituckSnd/CS_Stations/'
stations=['FRF_CS01-ADOP_Currents_ALL.csv','FRF_CS02-ADOP_Currents_ALL.csv','FRF_CS03-ADOP_Currents_ALL.csv',
         'FRF_CS04-ADOP_Currents_ALL.csv','FRF_CS05-ADOP_Currents_ALL.csv']
years=['2016','2017','2018']
months=['01','02','03','04','05','06','07','08','09','10','11','12']

for station in stations:
    print('for station:', station)
    dfw=pd.read_csv(file_path+station[4:8]+'/'+station, index_col='datetime')
    for year in years:
        print('   Processing year:', year)
        for month in months:
            start_date = year+'-'+month+'-01 00:00:00'
            end_date = year+'-'+month+'-31 59:59:59.9999999'

            #print(index, start_date, end_date)
            dfo=dfw.loc[(dfw.index >= start_date) & (dfw.index <= end_date)].copy()
            if dfo.empty:
                print('      No data for', month, year)
            else:
                dfo.to_csv(file_path+station[4:8]+'/'+'FRF_'+station[4:8]+'_ADOP_Currents_'+year+month+'.csv')
        
print('All done!')

for station: FRF_CS01-ADOP_Currents_ALL.csv
   Processing year: 2016
      No data for 01 2016
   Processing year: 2017
   Processing year: 2018
      No data for 02 2018
      No data for 03 2018
      No data for 04 2018
      No data for 05 2018
      No data for 06 2018
      No data for 07 2018
      No data for 08 2018
      No data for 09 2018
      No data for 10 2018
      No data for 11 2018
      No data for 12 2018
for station: FRF_CS02-ADOP_Currents_ALL.csv
   Processing year: 2016
      No data for 01 2016
      No data for 02 2016
   Processing year: 2017
   Processing year: 2018
      No data for 02 2018
      No data for 03 2018
      No data for 04 2018
      No data for 05 2018
      No data for 06 2018
      No data for 07 2018
      No data for 08 2018
      No data for 09 2018
      No data for 10 2018
      No data for 11 2018
      No data for 12 2018
for station: FRF_CS03-ADOP_Currents_ALL.csv
   Processing year: 2016
      No data for 01 2016
      No data for

In [5]:
s='FRF_CS01-ADOP_Currents_ALL.csv'

s[4:8]

'CS01'